In [1]:
import pyspark.sql.functions as F
from pyspark.sql import SparkSession

import getpass

# Load Custom Functions
try:
    from dags.bi_utils.sgp_lib import *
except:
    import sys
    sys.path.append('/home/jovyan/dags')
    from dags.bi_utils.sgp_lib import *

# Connect to spark

In [2]:
# Connect to spark
spark = (
    SparkSession
    .builder
    .master("spark://spark:7077")
    .config("spark.executor.memory", "12g")
    .appName("tpc")
    .getOrCreate()
)

In [ ]:
interactive=True
# Source
SOURCE_USER = '***'
SOURCE_PASS = '***'
SOURCE_SCHEMA = '***'
SOURCE_DB = '***'
SOURCE_HOST = '***'
SOURCE_PORT = '***'

# DL1
TARGET_USER = '***'
TARGET_PASS = '***'
TARGET_SCHEMA = '***'
TARGET_DB = '***'
TARGET_HOST = '***'
TARGET_PORT = '***'


In [3]:
SOURCE_USER = 'postgres'
SOURCE_PASS = 'postgres'
SOURCE_SCHEMA = 'tpc'
SOURCE_DB = 'tpc'
SOURCE_HOST = '192.168.1.30'
SOURCE_PORT = 5432

TARGET_USER = 'postgres'
TARGET_PASS = 'postgres'
TARGET_SCHEMA = 'tpc'
TARGET_DB = 'tpc'
TARGET_HOST = '192.168.1.31'
TARGET_PORT = 5432

In [4]:
# Connection string
source_string, source_properties = string_banco("postgresql", host=SOURCE_HOST, port=SOURCE_PORT, db=SOURCE_DB,
                                    user=SOURCE_USER, passw=SOURCE_PASS)

# List all tables in SOURCE

In [5]:
query = """SELECT
            relname AS table_name,
            n_live_tup AS row_count,
            pg_total_relation_size(relid) AS total_size_bytes
        FROM 
            pg_stat_user_tables
        WHERE 
            schemaname = 'tpc'"""
            
# A list with all tables in TPC schema
tb_tables_source = query_leitura(spark, source_properties, query)

# Checkpoint
tables_names = [row.table_name for row in tb_tables_source.collect()]
# row_count = [row.row_count for row in tb_tables_source.collect()]
# size_bytes = [row.total_size_bytes for row in tb_tables_source.collect()]
print(tables_names)

['finwire_sec', 'watchhistory', 'holdinghistory', 'prospect', 'trade', 'industry', 'finwire_fin', 'time', 'dailymarket', 'tradehistory', 'finwire_cmp', 'hr', 'statustype', 'tradetype', 'cashtransaction', 'taxrate']


# Create connection to TARGET

In [6]:
# Connect to TARGET
target_string, target_properties = string_banco("postgresql", host=TARGET_HOST, port=TARGET_PORT, db=TARGET_DB,
                                    user=TARGET_USER, passw=TARGET_PASS)

# Transfer tables from SOURCE to TARGET

In [7]:
# Columns to read
cols = ['*']
for table in tables_names:
    # Query
    query = f_query(cols, SOURCE_SCHEMA, table)
    print(query)
    # Load table
    tb_aux = query_leitura(spark, source_properties, query)
    # Write the data to the DL1
    # print(tb_aux.show(3, truncate=False))
    query_escrita(tb_aux, target_string, target_properties, TARGET_SCHEMA,
              table, "overwrite")

SELECT * FROM tpc.finwire_sec
SELECT * FROM tpc.watchhistory
SELECT * FROM tpc.holdinghistory
SELECT * FROM tpc.prospect
SELECT * FROM tpc.trade
SELECT * FROM tpc.industry
SELECT * FROM tpc.finwire_fin
SELECT * FROM tpc.time
SELECT * FROM tpc.dailymarket
SELECT * FROM tpc.tradehistory
SELECT * FROM tpc.finwire_cmp
SELECT * FROM tpc.hr
SELECT * FROM tpc.statustype
SELECT * FROM tpc.tradetype
SELECT * FROM tpc.cashtransaction
SELECT * FROM tpc.taxrate


In [ ]:
# Spark close session
spark.stop()